In [1]:
import findspark
findspark.init('D:\Program Files\Spark\spark-3.1.2-bin-hadoop2.7')

In [2]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
# dont forget to run vpn before run this code.

#df=spark.read.csv('hdfs://seneca1.lehre.hwr-berlin.de:8020/user/erik_eberhardt/yt_sample/*.csv').collect()
data = spark.read.format("csv").option("header", True).option("multiLine", True).option("inferSchema", True).load("hdfs://seneca1.lehre.hwr-berlin.de:8020/user/erik_eberhardt/yt_sample/*.csv")
data.show()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|trending_date|                tags|view_count|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|V1NK1lpEGSk|Season Two - Ep 1...|2021-06-13 10:07:43|UCQr98dS-5M_1xILT...|           superwog1|        23|     21.16.06|              [none]|   1350453|  96333|     597|         5885|https://i.ytimg.c...|         

In [4]:
sumlikes=data.groupBy("categoryId").sum("Likes").orderBy("sum(Likes)", ascending=False).show() 

+----------+----------+
|categoryId|sum(Likes)|
+----------+----------+
|        24| 208714654|
|        20| 144779419|
|        10| 133657338|
|        23| 107311547|
|        22|  91646005|
|        28|  25161227|
|        27|  20793198|
|        17|  13388555|
|         2|   6679282|
|        26|   3766880|
|         1|   1519279|
|        25|    816618|
|        19|    565515|
+----------+----------+



In [5]:
countCHID=data.groupBy("channelId").count().orderBy("count", ascending=False).show() 

+--------------------+-----+
|           channelId|count|
+--------------------+-----+
|UCUyfkq9e9ZfPzxOW...|   42|
|UClG8odDC8TS6Zpqk...|   42|
|UCF3K1Jf8hjFW8qli...|   42|
|UC7NCg0venpKJg3ku...|   28|
|UC2C6_mSbiLp3CplZ...|   28|
|UCke6I9N4KfC968-y...|   28|
|UC5Z7Fc_QmKPzHv1Z...|   28|
|UCw7FkXsC00lH2v2y...|   28|
|UCGIelM2Dj3zza3xy...|   28|
|UCvxfEIG3PHpgM0TM...|   28|
|UCvNBXWGykQrWb7kP...|   28|
|UCFB-QxUdWLN3jxmf...|   28|
|UC81hVmI5eEBIt3s3...|   28|
|UCT2X19JJaJGUN7mr...|   28|
|UCO4AcsPKEkIqDmbe...|   28|
|UCrqsNpKuDQZreGax...|   28|
|UC0jPwNs4B7yJySNe...|   28|
|UCymK_3BWUcoYVVf5...|   28|
|UCBVrn_SGKAOv3yKP...|   14|
|UCNdpD9V0PIrH1glU...|   14|
+--------------------+-----+
only showing top 20 rows



In [17]:
import pandas as pd
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, DecimalType, TimestampType
from decimal import Decimal
from pyspark import SparkContext

In [8]:
##convert list into dataframe
# Create a schema for the dataframe
schema = StructType ([
    StructField('video_id', StringType(), True),
    StructField('title', StringType(), True),
    StructField('publishedAt', TimestampType(), True),
    StructField('channelId', StringType(), True),
    StructField('channelTitle', StringType(), True),
    StructField('categoryId', StringType(), True),    
    StructField('trending_date', TimestampType(), True),    
    StructField('tags', StringType(), True),    
    StructField('view_count', DecimalType(), True),    
    StructField('likes', DecimalType(), True),    
    StructField('dislikes', DecimalType(), True),   
    StructField('comment_count', DecimalType(), True),   
    StructField('thumbnail_link', StringType(), True),
    StructField('comments_disabled', StringType(), True),
    StructField('ratings_disabled', StringType(), True),
    StructField('Description', StringType(), True)
])

spark = SparkSession.builder.appName("DFpyspark").getOrCreate()
sc=spark.sparkContext

df = spark.read.csv("hdfs://seneca1.lehre.hwr-berlin.de:8020/user/erik_eberhardt/yt_sample/*.csv", header=True,sep=",",schema=schema)
df.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|            video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|trending_date|                tags|view_count|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         Description|
+--------------------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|         V1NK1lpEGSk|Season Two - Ep 1...|2021-06-13 10:07:43|UCQr98dS-5M_1xILT...|           superwog1|        23|         null|              [none]|   1350453|  96333|     597|        

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm 
from pyspark import SparkConf
from subprocess import check_output
import re, string
from operator import add


matplotlib.rcParams['figure.figsize'] = (10,10)

In [20]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [21]:
sc.master

'local[*]'

In [32]:
import csv 
from io import StringIO 
from collections import namedtuple
from pyspark.sql.functions import *
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import glob
import seaborn as sns

In [10]:
data.count()

2800

In [11]:
data.take(1)

[Row(video_id='V1NK1lpEGSk', title='Season Two - Ep 1 The P Plates', publishedAt=datetime.datetime(2021, 6, 13, 10, 7, 43), channelId='UCQr98dS-5M_1xILTm4HAk-A', channelTitle='superwog1', categoryId=23, trending_date='21.16.06', tags='[none]', view_count=1350453, likes=96333, dislikes=597, comment_count=5885, thumbnail_link='https://i.ytimg.com/vi/V1NK1lpEGSk/default.jpg', comments_disabled=False, ratings_disabled=False, description='All 6 episodes available now free on ABC iview! Stream now: https://abc.net.au/superwog')]

In [33]:
import pyspark

In [34]:
dir(pyspark)

['Accumulator',
 'AccumulatorParam',
 'BarrierTaskContext',
 'BarrierTaskInfo',
 'BasicProfiler',
 'Broadcast',
 'HiveContext',
 'InheritableThread',
 'MarshalSerializer',
 'PickleSerializer',
 'Profiler',
 'RDD',
 'RDDBarrier',
 'Row',
 'SQLContext',
 'SparkConf',
 'SparkContext',
 'SparkFiles',
 'SparkJobInfo',
 'SparkStageInfo',
 'StatusTracker',
 'StorageLevel',
 'TaskContext',
 '_NoValue',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_globals',
 'accumulators',
 'broadcast',
 'cloudpickle',
 'conf',
 'context',
 'copy_func',
 'files',
 'find_spark_home',
 'java_gateway',
 'join',
 'keyword_only',
 'profiler',
 'rdd',
 'rddsampler',
 'resource',
 'resultiterable',
 'serializers',
 'shuffle',
 'since',
 'sql',
 'statcounter',
 'status',
 'storagelevel',
 'taskcontext',
 'traceback_utils',
 'types',
 'util',
 'version',
 'wraps']

In [35]:
from pyspark.sql import SQLContext 

In [36]:
sqlContext = SQLContext(sc)


In [37]:
data.registerTempTable('ytTable')

In [38]:
#make queries
sqlContext.sql('SELECT * FROM ytTable').show()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|trending_date|                tags|view_count|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|V1NK1lpEGSk|Season Two - Ep 1...|2021-06-13 10:07:43|UCQr98dS-5M_1xILT...|           superwog1|        23|     21.16.06|              [none]|   1350453|  96333|     597|         5885|https://i.ytimg.c...|         

In [45]:
import pandas as pd
import numpy as np
import os
import requests
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mc
import seaborn as sns
from IPython.display import HTML
import re
from collections import Counter
import wordcloud
from pprint import pprint
import random
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk import download as nltk_download
from PIL import Image
import emoji
import tempfile
import io
from PIL import Image
import time
from scipy.spatial import distance
from skimage import color as skcolor
#from imageai.Detection import ObjectDetection

In [51]:
sqlContext.sql('SELECT min(view_count) FROM ytTable').show()

+---------------+
|min(view_count)|
+---------------+
|          62051|
+---------------+



In [72]:
data.sort('view_count').show()
#df.sort_values(by='views', ascending=False).drop_duplicates(subset=['video_id']).head()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-----+--------+-------------+--------------------+-----------------+----------------+--------------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|trending_date|                tags|view_count|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-----+--------+-------------+--------------------+-----------------+----------------+--------------------+
|mXxSeVG9uQk|Real Deal gets a ...|2021-06-09 07:34:52|UCST3-8pPhHzJeRsX...|Castlemaine Rod Shop|         2|     21.16.06|REAL DEAL|HOLDEN|...|     62051| 4575|      53|          580|https://i.ytimg.c...|            false

In [82]:
data.filter(data.video_id == 'mXxSeVG9uQk').show()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-----+--------+-------------+--------------------+-----------------+----------------+--------------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|trending_date|                tags|view_count|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-----+--------+-------------+--------------------+-----------------+----------------+--------------------+
|mXxSeVG9uQk|Real Deal gets a ...|2021-06-09 07:34:52|UCST3-8pPhHzJeRsX...|Castlemaine Rod Shop|         2|     21.16.06|REAL DEAL|HOLDEN|...|     62459| 4584|      53|          580|https://i.ytimg.c...|            false

In [79]:
#drop all duplicate data by video_id
df1=data.dropDuplicates(['video_id'])

In [81]:
#check sample duplicate data. is it taking the newest data? yes (compare to above code)
df1.filter(data.video_id == 'mXxSeVG9uQk').show()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-----+--------+-------------+--------------------+-----------------+----------------+--------------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|trending_date|                tags|view_count|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-----+--------+-------------+--------------------+-----------------+----------------+--------------------+
|mXxSeVG9uQk|Real Deal gets a ...|2021-06-09 07:34:52|UCST3-8pPhHzJeRsX...|Castlemaine Rod Shop|         2|     21.16.06|REAL DEAL|HOLDEN|...|     62459| 4584|      53|          580|https://i.ytimg.c...|            false

In [83]:
#save the final csv into local 
df1.write.format('csv').option('header','true').save('E:/Github/youtube/compile')